# SQLite DB 생성 프로그램

In [1]:
from sqlalchemy import create_engine
import pandas as pd
import binascii   # 한글 변환에 필요한 라이브러리
import sys
import sqlite3

## 변수지정

In [2]:
N_season = '23N'
F_season = '22F'
S_season = '23S'

## Oracle DB 연결

In [3]:
sys.path.append('/settings')
import config

def connect_db(sid: str):
    if sid != config.COMPANY_DB_CONFIG['sid']:
        raise ValueError("DB 를 찾을 수 없습니다.")
    
    conn = create_engine(
        "oracle+cx_oracle://{user}:{password}@{host}:{port}/{sid}?encoding=UTF-8&nencoding=UTF-8".format(
            user=config.COMPANY_DB_CONFIG['user'],
            password=config.COMPANY_DB_CONFIG['password'],
            host=config.COMPANY_DB_CONFIG['host'],
            port=config.COMPANY_DB_CONFIG['port'],
            sid=config.COMPANY_DB_CONFIG['sid']
        )
    )
    return conn

engine = connect_db('IVY')

## SQL문

### 1. I_SUJU_MASTER_DAILY_T

- 매일기록용, SQLITE DB 전용 테이블

In [4]:
sql1: str = f'''
SELECT (SELECT SYSDATE FROM DUAL) query_date,
       master_order,
       master_year,
       master_season,
       master_school,
       master_agent,
       master_bokjong,
       master_ser,
       master_tkyk,
       master_person,
       master_suju_date,
       master_suju_qty,
       master_jisi_qty,
       master_status,
       master_hold,
       master_remark,
       master_remake,
       master_jaepum,
       master_taip_date,
       master_hold_date,
       master_quota,
       master_sex,
       master_squota,
       master_prodm_qty,
       master_prodm_date,
       master_appv_end_dt,
       master_appv_start_dt,
       master_st20_dt,
       master_st03_date,
       master_st00_dt,
       master_st04_date,
       master_ns_quota,
       master_grade_cnt,
       master_suju_seq
FROM   i_suju_master_t
WHERE  master_quota IN ( '{N_season}', '{S_season}', '{F_season}' )
       AND master_tkyk IN ( 'C', 'D', 'H', 'I', 'L', 'R' )
       AND master_status >= '03'
       AND master_status <= '60' 
'''

### 2. I_SCH_T

- 원본과 거의 동일하나 필요없는 항목은 빠져있다.

In [5]:
sql2: str = f'''
SELECT sch_code,
       Rawtohex(utl_raw.Cast_to_raw(sch_name)) sch_name,
       sch_tel, sch_post, sch_qty, sch_gbn, sch_suit_yn, sch_tkyk, sch_spec, sch_agen,
       sch_type, sch_del_yn, sch_sale_yn, sch_f_bokjong, sch_season, sch_com_cd, sch_com_gb,
       sch_gb_cd, sch_style_gb, sch_end_gb, sch_stand_gb, sch_combok_gb, sch_s_add_bokjong, sch_incheon_gb, sch_cd_area_gb, sch_cd_sch_gb
FROM   i_sch_t
WHERE  1=1
'''

### 3. I_AGEN_T

- 원본과 거의 동일하나 필요없는 항목은 빠져있다.

In [6]:
sql3: str = f'''
SELECT agen_code,
       Rawtohex(utl_raw.Cast_to_raw(agen_name)) agen_name,
       agen_area,
       agen_tkyk,
       Rawtohex(utl_raw.Cast_to_raw(agen_president)) agen_president,
       Rawtohex(utl_raw.Cast_to_raw(agen_store)) agen_store,
       agen_no,
       agen_post,
       Rawtohex(utl_raw.Cast_to_raw(agen_addr)) agen_addr,
       agen_tel,
       Rawtohex(utl_raw.Cast_to_raw(agen_store1)) agen_store1,
       Rawtohex(utl_raw.Cast_to_raw(agen_saddr1)) agen_saddr1,
       agen_presi_tel,
       agen_presi_ctel,
       agen_mailaddr,
       agen_jaepum,
       Rawtohex(utl_raw.Cast_to_raw(agen_store1)) agen_store5,
       Rawtohex(utl_raw.Cast_to_raw(agen_store1)) agen_saddr5,
       agen_del_yn
FROM   i_agen_t
WHERE  1=1
'''

### 4. I_TKYK_T

- 원본과 거의 동일하나 필요없는 항목은 빠져있다.

In [7]:
sql4: str = f'''
SELECT tkyk_code,
       Rawtohex(utl_raw.Cast_to_raw(tkyk_name)) tkyk_name
FROM   i_tkyk_t
WHERE  1=1
'''

## 함수 모음

### 한글변환 함수

In [8]:
# US7ASCII -> CP949(완성형한글) 로 변환
def us7ascii_to_cp949(df: pd.DataFrame) -> pd.DataFrame:
    for index, byte_data in enumerate(df):
        if byte_data == None: # null 값이면 패스. 안하면 변환 에러난다.
            continue
        byte_data = binascii.unhexlify(df[index])  # 16진수 문자열 hexstr로 표현된 바이너리 데이터를 반환. 역함수는 b2a_hex()
        df[index] = byte_data.decode("cp949")  # 바이트 변환값 -> cp949(완성형 한글) 로 변환
    return df

### 쿼리 함수

In [9]:
def oracle_query(oracleSql: str) -> pd.DataFrame:
    df = pd.read_sql_query(oracleSql, engine)
    
    if 'tkyk_name' in df.columns: # 해당컬럼이 없어도 에러없이 처리
        df_temp = df['tkyk_name'].copy()
        df['tkyk_name'] = us7ascii_to_cp949(df_temp)

    if 'agen_name' in df.columns:
        df_temp = df['agen_name'].copy()
        df['agen_name'] = us7ascii_to_cp949(df_temp)
        
    if 'agen_president' in df.columns:
        df_temp = df['agen_president'].copy()
        df['agen_president'] = us7ascii_to_cp949(df_temp)
        
    if 'agen_store' in df.columns:
        df_temp = df['agen_store'].copy()
        df['agen_store'] = us7ascii_to_cp949(df_temp)
        
    if 'agen_addr' in df.columns:
        df_temp = df['agen_addr'].copy()
        df['agen_addr'] = us7ascii_to_cp949(df_temp)
        
    if 'agen_store1' in df.columns:
        df_temp = df['agen_store1'].copy()
        df['agen_store1'] = us7ascii_to_cp949(df_temp)
        
    if 'agen_saddr1' in df.columns:
        df_temp = df['agen_saddr1'].copy()
        df['agen_saddr1'] = us7ascii_to_cp949(df_temp)
        
    if 'agen_store5' in df.columns:
        df_temp = df['agen_store5'].copy()
        df['agen_store5'] = us7ascii_to_cp949(df_temp)
        
    if 'agen_saddr5' in df.columns:
        df_temp = df['agen_saddr5'].copy()
        df['agen_saddr5'] = us7ascii_to_cp949(df_temp)

    if 'sch_name' in df.columns:
        df_temp = df['sch_name'].copy()
        df['sch_name'] = us7ascii_to_cp949(df_temp)
    return df

In [10]:
df1 = oracle_query(sql1)
df2 = oracle_query(sql2)
df3 = oracle_query(sql3)
df4 = oracle_query(sql4)

In [11]:
df1.head()

,query_date,master_order,master_year,master_season,master_school,master_agent,master_bokjong,master_ser,master_tkyk,master_person,...,master_prodm_date,master_appv_end_dt,master_appv_start_dt,master_st20_dt,master_st03_date,master_st00_dt,master_st04_date,master_ns_quota,master_grade_cnt,master_suju_seq
0,2022-11-10 17:32:56,22FTD105B 1,22,F,TD105,VT135,B,1,I,VT135,...,2022-07-14,2022-06-09,None,2022-06-10 14:23:04,2022-04-29 13:20:04,None,2022-06-09 13:41:15,23F,N,1
1,2022-11-10 17:32:56,22FTD105R 1,22,F,TD105,VT135,R,1,I,VT135,...,2022-07-30,2022-06-09,None,2022-06-10 14:23:05,2022-04-29 13:20:04,None,2022-06-09 13:41:15,23F,N,1
2,2022-11-10 17:32:56,22FTB008B 1,22,F,TB008,VT135,B,1,I,VT135,...,2022-07-27,2022-06-28,None,2022-06-30 13:43:16,2022-04-29 13:20:04,None,2022-06-28 15:38:50,23F,N,1
3,2022-11-10 17:32:56,22FTB008R 1,22,F,TB008,VT135,R,1,I,VT135,...,2022-07-30,2022-06-29,None,2022-06-30 13:43:18,2022-04-29 13:20:04,None,2022-06-29 11:19:17,23F,N,1
4,2022-11-10 17:32:56,22FTB008V 1,22,F,TB008,VT135,V,1,I,VT135,...,2022-09-19,2022-06-29,None,2022-06-30 13:43:18,2022-04-29 13:20:05,None,2022-06-29 11:19:17,23F,N,1


## SQLITE 시작

In [12]:
connect = sqlite3.connect('./data/daliy_order.db') # conn 객체 생성 = DB 파일명.

In [13]:
cursor = connect.cursor() # 커서 생성

In [14]:
df1.to_sql('I_SUJU_MASTER_DAILY_T', connect, index_label='num', if_exists='append') # 테이블이 존재할 경우 데이터만 넣는다.

# df1.to_sql('I_SUJU_MASTER_DAILY_T', connect, index_label='num', if_exists='replace') # 테이블이 존재할 경우 테이블 삭제 후 재생성

df2.to_sql('I_SCH_T', connect, index=False, if_exists='replace') # 테이블이 존재할 경우 테이블 삭제 후 재생성
df3.to_sql('I_AGEN_T', connect, index=False, if_exists='replace') # 테이블이 존재할 경우 테이블 삭제 후 재생성
df4.to_sql('I_TKYK_T', connect, index=False, if_exists='replace') # 테이블이 존재할 경우 테이블 삭제 후 재생성

16

In [15]:
connect.close()